In [ ]:
# ==============================================================================
# PART 1: INSTALL REQUIRED LIBRARIES
# ==============================================================================
print("⏳ Installing required libraries...")
!pip install gradio groq huggingface_hub -q
print("✅ Libraries installed.")

# ==============================================================================
# PART 2: IMPORTS, API KEY SETUP, AND AUTHENTICATION
# ==============================================================================
import gradio as gr
import os
from groq import Groq
from huggingface_hub import InferenceClient, HfFolder
from google.colab import userdata

print("🔑 Setting up API keys and authentication...")

# Securely get the Groq API key
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY
    print("✅ Groq API Key loaded.")
except Exception as e:
    print(f"⚠️ Could not load Groq API Key. Please add it to Colab Secrets. Error: {e}")

# Securely get the Hugging Face token and authenticate
try:
    HF_TOKEN = userdata.get('HUGGING_FACE_API_KEY')
    HfFolder.save_token(HF_TOKEN)
    print("✅ Hugging Face Token loaded and authenticated.")
except Exception as e:
    print(f"⚠️ Could not load Hugging Face Token. Please add it to Colab Secrets. Error: {e}")


# ==============================================================================
# PART 3: INITIALIZE API CLIENTS (This connects to cloud services)
# ==============================================================================
print("☁️ Initializing API clients...")

# 1. Groq Client (for Chatbot Tab)
try:
    groq_client = Groq()
    print("✅ Groq client initialized.")
except Exception as e:
    print(f"🔥 Error initializing Groq client: {e}")

# 2. Hugging Face Inference Client (for everything else)
# This client will call the models running on Hugging Face's servers.
try:
    inference_client = InferenceClient(token=HF_TOKEN)
    print("✅ Hugging Face Inference client initialized.")
except Exception as e:
    print(f"🔥 Error initializing Hugging Face client: {e}")

# Define the models we will call on Hugging Face's servers
IMAGE_MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"
TRANSLATION_MODEL_ID = "facebook/nllb-200-distilled-600M"
TRANSCRIPTION_MODEL_ID = "openai/whisper-base"


# ==============================================================================
# PART 4: DEFINE CORE FUNCTIONS (Using API calls)
# ==============================================================================

# --- Function for Tab 1: Audio Processing ---
def process_audio_and_generate(audio_path, generate_image_checkbox):
    if not audio_path:
        gr.Warning("No audio file provided. Please upload an audio file.")
        return "", "", None

    transcription, english_text, image_output = "", "", None

    # 1. Transcribe audio to text via API call
    try:
        print("🎤 Calling HF API for transcription...")
        transcription_result = inference_client.automatic_speech_recognition(
            audio=audio_path,
            model=TRANSCRIPTION_MODEL_ID
        )
        transcription = transcription_result['text']
    except Exception as e:
        gr.Error(f"Transcription failed! The model might be loading. Please try again. Error: {e}")
        return "Transcription Failed", "", None

    # 2. Translate text to English via API call
    try:
        print("🌐 Calling HF API for translation...")
        translation_result = inference_client.translation(
            text=transcription,
            src_lang="tam_Taml",
            tgt_lang="eng_Latn",
            model=TRANSLATION_MODEL_ID
        )
        english_text = translation_result[0]['translation_text']
    except Exception as e:
        gr.Error(f"Translation failed! Error: {e}")
        return transcription, "Translation Failed", None

    # 3. Generate image via API call if checkbox is ticked
    if generate_image_checkbox:
        try:
            print("🎨 Calling HF API for image generation...")
            image_output = inference_client.text_to_image(english_text, model=IMAGE_MODEL_ID)
        except Exception as e:
            gr.Warning(f"Image generation failed! The model might be loading. Error: {e}")

    return transcription, english_text, image_output

# --- Function for Tab 2: Prompt to Image ---
def generate_image_from_prompt(prompt):
    if not prompt or not prompt.strip():
        gr.Warning("Prompt is empty. Please enter some text.")
        return None
    try:
        print(f"🎨 Calling HF API for image generation from prompt: '{prompt}'")
        image = inference_client.text_to_image(prompt, model=IMAGE_MODEL_ID)
        return image
    except Exception as e:
        gr.Error(f"Image generation failed! The model might be loading. Please try again. Error: {e}")
        return None

# --- Function for Tab 3: Groq Chatbot ---
def chatbot_response(message, history):
    print("💬 Generating chatbot response with Groq...")
    history_groq_format = [{"role": "system", "content": "You are a helpful assistant."}]
    for human, assistant in history:
        history_groq_format.append({"role": "user", "content": human})
        history_groq_format.append({"role": "assistant", "content": assistant})
    history_groq_format.append({"role": "user", "content": message})

    try:
        chat_completion = groq_client.chat.completions.create(
            messages=history_groq_format,
            model="moonshotai/kimi-k2-instruct"
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        gr.Error(f"Chatbot failed! Error: {e}")
        return f"Sorry, I encountered an error: {e}"


# ==============================================================================
# PART 5: BUILD THE GRADIO USER INTERFACE
# ==============================================================================
print("🚀 Building the Gradio interface...")

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 All-in-One AI Suite (Cloud API Version)")

    with gr.Tabs():
        # --- Tab 1: Audio Processing ---
        with gr.TabItem("🎙️ Audio to Text & Image"):
            with gr.Row():
                with gr.Column(scale=1):
                    audio_input = gr.Audio(type="filepath", label="Upload Tamil Audio (பேச்சு)")
                    audio_image_checkbox = gr.Checkbox(label="Generate Image from Translation?")
                    audio_button = gr.Button("Process Audio", variant="primary")
                with gr.Column(scale=2):
                    transcription_output = gr.Textbox(label="Transcription (Tamil)", interactive=False)
                    translation_output = gr.Textbox(label="Translation (English)", interactive=False)
                    audio_image_output = gr.Image(label="Generated Image (படம்)")

        # --- Tab 2: Prompt to Image ---
        with gr.TabItem("🖼️ Prompt to Image"):
            with gr.Row():
                with gr.Column(scale=2):
                    image_prompt_input = gr.Textbox(label="Enter your prompt", lines=4, placeholder="e.g., A majestic lion in the savanna at sunset...")
                    image_button = gr.Button("Generate Image", variant="primary")
                with gr.Column(scale=1):
                    image_output = gr.Image(label="Generated Image")

        # --- Tab 3: Chatbot ---
        with gr.TabItem("💬 Groq Chatbot"):
            gr.ChatInterface(
                chatbot_response,
                title="AI Chatbot",
                description="Ask me anything! Powered by Groq and Llama 3.",
                examples=[["Hello!"], ["What is the capital of India?"]]
            )

    # --- Define button click actions ---
    audio_button.click(
        fn=process_audio_and_generate,
        inputs=[audio_input, audio_image_checkbox],
        outputs=[transcription_output, translation_output, audio_image_output]
    )

    image_button.click(
        fn=generate_image_from_prompt,
        inputs=image_prompt_input,
        outputs=image_output
    )

# ==============================================================================
# PART 6: LAUNCH THE APPLICATION
# ==============================================================================
print("🎉 Launching the application! Click the public URL to open.")
demo.launch(share=True, debug=True)

⏳ Installing required libraries...
✅ Libraries installed.
🔑 Setting up API keys and authentication...
✅ Groq API Key loaded.
✅ Hugging Face Token loaded and authenticated.
☁️ Initializing API clients...
✅ Groq client initialized.
✅ Hugging Face Inference client initialized.
🚀 Building the Gradio interface...


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


🎉 Launching the application! Click the public URL to open.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4dda698355e9c707bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
